# Notebook 2 : Pandas avancé

In [ ]:
import pandas as pd

## Tennis

Nous considérons les données des résultats des matchs de tennis masculin des tournois de Roland Garros et Wimbledon en 2013. La liste des variables et leur signification se trouvent sur [cette page](https://archive.ics.uci.edu/dataset/300/tennis+major+tournament+match+statistics) dans la section *Additional Variable Information*.

1. Commencer par charger le jeu de données relatif au tournoi de Roland Garros dans un dataframe `rg` à partir du fichier `rolandgarros2013.csv`.

In [ ]:
rg = pd.read_csv("data/rolandgarros2013.csv")
rg

2. Afficher les noms des demi-finalistes.

In [ ]:
rg[rg.Round == 6].filter(["Player1", "Player2"])

3. Calculer le nombre moyen d'aces par match dans le tournoi.

In [ ]:
(rg["ACE.1"] + rg["ACE.2"]).mean()

4. Combien y a-t-il eu d'aces par match en moyenne à chaque niveau du tournoi ?

In [ ]:
(rg["ACE.1"] + rg["ACE.2"]).groupby(rg.Round).mean()

5. Filtrer les matchs pour lesquels au moins une des variables `DBF.1` et `DBF.2` est manquante.

In [ ]:
rg[rg["DBF.1"].isna() | rg["DBF.2"].isna()]

6. Remplacer les valeurs manquantes de `DBF.1` par zéro avec la méthode `loc`.

In [ ]:
# La commande suivante ne fonctionne pas, il faut utiliser loc
# rg[rg["DBF.1"].isna()]["DBF.1"] = 0

In [ ]:
rg.loc[rg["DBF.1"].isna(), "DBF.1"] = 0

7. Remplacer les valeurs manquantes de `DBF.2` par zéro avec la méthode `fillna`.

In [ ]:
rg["DBF.2"] = rg["DBF.2"].fillna(0)

8. Extraire la liste des participants à partir des colonnes `Player1` et `Player2`. Une façon de faire consiste à utiliser `concat` et la méthode `drop_duplicates` pour obtenir le résultat sous la forme d'une série et de la convertir en dataframe avec la méthode `to_frame`.

In [ ]:
joueurs = (
    pd.concat([rg.Player1, rg.Player2])
    .drop_duplicates()
    .to_frame(name="joueur")
)
joueurs

9. Écrire une fonction `n_match` qui prend une chaîne de caractères `joueur` en entrée et retourne le nombre de matchs disputés par le joueur.

In [ ]:
def n_match(joueur):
    return len(rg[rg.Player1 == joueur]) + len(rg[rg.Player2 == joueur])

n_match("Roger Federer")

10. Utiliser les deux question précédentes et la méthode `apply` pour compter le nombre de matchs que chaque participant a disputé et ordonner le résultat par ordre décroissant.

In [ ]:
joueurs["n_match"] = joueurs.joueur.apply(n_match)
joueurs.sort_values(by="n_match", ascending=False, inplace=True)
joueurs

11. Charger maintenant le jeu de données relatif au tournoi de Wimbledon dans un dataframe `wb` à partir du fichier `wimbledon2013.csv`.

In [ ]:
wb = pd.read_csv("data/wimbledon2013.csv")
wb

12. Ajouter une colonne `Tournoi` dans les dataframes `rg` et `wb` contenant respectivement les chaînes de caractères `"RG"` et `"WB"`.

In [ ]:
rg["Tournoi"] = "RG"
wb["Tournoi"] = "WB"

13. Concaténer les deux dataframes dans un nouveau dataframe `tennis`.

In [ ]:
tennis = pd.concat([rg, wb], ignore_index=True)

# Le même résultat peut être obtenu sans la création préalable de la colonne Tournoi.
# Cela demande de bien comprendre la fonction concat et ses paramètres keys et names.
# tennis = (
#     pd.concat(
#         [
#             # Supprime la colonne Tournoi de la question précédente
#             rg.drop(columns=["Tournoi"]),
#             wb.drop(columns=["Tournoi"])
#         ],
#         keys=["RG", "WB"],
#         names=["Tournoi", "Index"]
#     )
#     .reset_index()
#     .drop(columns=["Index"])
# )

tennis

14. Utiliser le dataframe `tennis` pour comparer le nombre moyen d'aces par match à chaque niveau du tournoi à Roland Garros et à Wimbledon. Afficher le résultat en format large.

In [ ]:
(
    (tennis["ACE.1"] + tennis["ACE.2"])
    .to_frame(name="Aces")
    .groupby([tennis.Tournoi, tennis.Round])
    .mean()
    .reset_index()
    .pivot(index="Tournoi", columns="Round", values="Aces")
)

15. Quelle différence y a-t-il dans le format des noms des joueurs entre les dataframes `rg` et `wb` ?

In [ ]:
# Le prénom des joueurs est limité à son initiale (e.g. "Roger Federer" devient "R.Federer")

print("*** ROLAND GARROS ***")
print(rg[rg.Player2.str.contains("Federer")].Player2)

print("*** WIMBLEDON ***")
print(wb[wb.Player2.str.contains("Federer")].Player2)

16. Construire un dataframe `rg_victoires` avec les trois colonnes suivantes pour le tournoi de Roland Garros :
- `joueur` : nom du joueur tel qu'il est donné dans `rg`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

In [ ]:
# Liste des joueurs
rg_victoires = (
    pd.concat([rg.Player1, rg.Player2])
    .drop_duplicates()
    .to_frame(name="joueur")
)

# Extraction du nom de famille
rg_victoires["nom_joueur"] = (
    rg_victoires.joueur.str.split()
    .apply(lambda v: v[-1]) # Alternative: .str.get(-1)
)

# Nombre de victoires
def n_victoire(df, joueur):
    return (
        len(df[(df.Player1 == joueur) & (df.Result == 1)])
        + len(df[(df.Player2 == joueur) & (df.Result == 0)])
    )

rg_victoires["n_victoire"] = (
    rg_victoires.joueur
    .apply(lambda joueur: n_victoire(rg, joueur))
)

rg_victoires.sort_values(by="n_victoire", ascending=False)

17. Construire un dataframe `wb_victoires` avec les trois colonnes suivantes pour le tournoi de Wimbledon :
- `joueur` : nom du joueur tel qu'il est donné dans `wb`,
- `nom_joueur` : nom de famille du joueur uniquement,
- `n_victoire` : nombre de matchs gagnés dans le tournoi.

In [ ]:
# Liste des joueurs
wb_victoires = (
    pd.concat([wb.Player1, wb.Player2])
    .drop_duplicates()
    .to_frame(name="joueur")
)

# Extraction du nom de famille
wb_victoires["nom_joueur"] = (
    wb_victoires.joueur.str.split(".") # On utilise le point comme séparateur
    .apply(lambda v: v[-1]) # Alternative: .str.get(-1)
)

# Nombre de victoires
wb_victoires["n_victoire"] = (
    wb_victoires.joueur
    .apply(lambda joueur: n_victoire(wb, joueur))
)

wb_victoires.sort_values(by="n_victoire", ascending=False)

18. Faire une jointure entre `rg_victoires` et `wb_victoires` sur la colonne `nom_joueur` pour comparer le nombre de victoires par tournoi pour chaque joueur. Expliquer la différence de résultat selon que la jointure est à gauche, à droite, intérieure ou extérieure.

In [ ]:
# Jointure à gauche : joueurs au moins présents à Roland Garros mais pas nécessairement à Wimbledon
(
    rg_victoires.filter(["nom_joueur", "n_victoire"])
    .merge(
        wb_victoires.filter(["nom_joueur", "n_victoire"]),
        how="left",
        on="nom_joueur",
        suffixes=["_rg", "_wb"], # Utilisation des suffixes
    )
    .sort_values(by=["n_victoire_rg", "n_victoire_wb"], ascending=False)
)

In [ ]:
# Jointure à droite : joueurs au moins présents à Wimbledon mais pas nécessairement à Roland Garros
(
    rg_victoires.filter(["nom_joueur", "n_victoire"])
    .merge(
        wb_victoires.filter(["nom_joueur", "n_victoire"]),
        how="right",
        on="nom_joueur",
        suffixes=["_rg", "_wb"], # Utilisation des suffixes
    )
    .sort_values(by=["n_victoire_wb", "n_victoire_rg"], ascending=False)
)

In [ ]:
# Jointure intérieure : joueurs présents à Roland Garros et à Wimbledon
(
    rg_victoires.filter(["nom_joueur", "n_victoire"])
    .merge(
        wb_victoires.filter(["nom_joueur", "n_victoire"]),
        how="inner",
        on="nom_joueur",
        suffixes=["_rg", "_wb"], # Utilisation des suffixes
    )
    .sort_values(by=["n_victoire_rg", "n_victoire_wb"], ascending=False)
)

In [ ]:
# Jointure extérieure : joueurs présents à Roland Garros ou à Wimbledon
(
    rg_victoires.filter(["nom_joueur", "n_victoire"])
    .merge(
        wb_victoires.filter(["nom_joueur", "n_victoire"]),
        how="outer",
        on="nom_joueur",
        suffixes=["_rg", "_wb"], # Utilisation des suffixes
    )
    .sort_values(by=["n_victoire_rg", "n_victoire_wb"], ascending=False)
)